# Lesson: Usando Pydantic com OpenAI

Nesta lição vamos aprender como usar **modelos Pydantic** para estruturar e validar saídas diretamente de LLMs da OpenAI.

Objetivos:
- Eliminar parsing manual de strings.
- Garantir respostas estruturadas e validadas.
- Construir **data models claros** que refletem os resultados esperados.
- Usar as bibliotecas modernas `instructor` e `pydantic_ai` para simplificar o processo.

## 1. Setup

Instale as bibliotecas necessárias:

```bash
pip install openai pydantic instructor pydantic-ai
```

E configure sua **OpenAI API Key** como variável de ambiente:

```bash
export OPENAI_API_KEY="sua_chave_aqui"
```

In [ ]:
import os

from openai import OpenAI

# Inicializa o cliente OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

## 2. Exemplo com Pydantic + Instructor

Aqui vamos criar um modelo de dados `Person` e pedir ao LLM para preencher com informações.
A biblioteca `instructor` permite passar o modelo Pydantic diretamente para o cliente OpenAI.

In [ ]:
import instructor
from pydantic import BaseModel


# Definindo um modelo de dados
class Person(BaseModel):
    name: str
    age: int
    city: str


# Habilita instructor no cliente
client = instructor.patch(client)

# Solicita ao modelo
person = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": "Crie uma pessoa fictícia com nome, idade e cidade.",
        }
    ],
    response_model=Person,
)

print(person)

## 3. Exemplo com Pydantic_AI

A biblioteca `pydantic_ai` oferece uma forma ainda mais simplificada de conectar Pydantic + LLMs.
Aqui, pediremos ao modelo uma lista estruturada de **livros recomendados**.

In [ ]:
from pydantic_ai import Agent


class Book(BaseModel):
    title: str
    author: str
    year: int


# Criando um agente baseado no modelo gpt-4o
agent = Agent("gpt-4o", response_format=list[Book])

books = agent.run("Liste 3 livros de ficção científica importantes.")

for b in books:
    print(b)

## 4. Exemplo prático: Sugestões de restaurantes

Aqui criamos um modelo mais elaborado que retorna uma lista estruturada de restaurantes sugeridos pelo modelo.

In [ ]:
class Restaurant(BaseModel):
    name: str
    cuisine: str
    rating: float
    city: str


agent = Agent("gpt-4o", response_format=list[Restaurant])

restaurants = agent.run(
    "Sugira 3 restaurantes famosos em São Paulo com nome, tipo de cozinha, nota (0-5) e cidade."
)

for r in restaurants:
    print(r)

## 5. Conclusão

- Usando **Pydantic** + **LLMs**, conseguimos saídas já estruturadas e validadas.
- Com `instructor`, passamos o modelo Pydantic direto no `response_model`.
- Com `pydantic_ai`, definimos o esquema de saída no próprio agente.
- Benefícios: código mais robusto, fácil de manter e com menos erros de parsing manual.